I calculate the number of local with the same activity within a radius

In [1]:
import pandas as pd
import clean_functions as cl

In [2]:
df_locals_short = pd.read_csv('Data/censolocales/locals_sh.csv')

In [3]:
df_locals_short.columns

Index(['id_local', 'id_distrito_local', 'desc_distrito_local',
       'id_barrio_local', 'desc_barrio_local', 'desc_tipo_acceso_local',
       'id_situacion_local', 'desc_situacion_local', 'clase_vial_acceso',
       'desc_vial_acceso', 'nom_acceso', 'num_acceso', 'cal_acceso',
       'id_agrupacion', 'nombre_agrupacion', 'id_tipo_agrup',
       'desc_tipo_agrup', 'rotulo', 'id_seccion', 'desc_seccion',
       'id_division', 'desc_division', 'id_epigrafe', 'desc_epigrafe', 'conc',
       'desc_sit_loc_modif_19', 'desc_sit_loc_modif_18',
       'desc_sit_loc_modif_17', 'desc_sit_loc_modif_16',
       'desc_sit_loc_modif_15', 'cerrado_19', 'abierto_19', 'cerrado_18',
       'abierto_18', 'cerrado_17', 'abierto_17', 'cerrado_16', 'abierto_16',
       'target', 'desc_act_norm', 'id_act_norm', 'ab_17_19', 'loc_dist_act',
       'ab_dist_act_17_19', 'total_loc_act', 'total_ab_act_17_19', 'loc_dist',
       'ab_dist_17_19', 'loc_na_dist', 'ab_dist_act_17_19_rate',
       'total_ab_act_17_19_r

In [4]:
df_locals_short = df_locals_short[['id_local','desc_epigrafe','lat','lon']]

In [5]:
df_locals_short.columns

Index(['id_local', 'desc_epigrafe', 'lat', 'lon'], dtype='object')

In [6]:
from geopy.distance import geodesic

def points_in_radius(point1, point2,radius):
    return geodesic(point1, point2).meters < radius

df_ = df_locals_short.copy()
df_['point'] = list(zip(df_.loc[:,'lat'],df_.loc[:,'lon']))
df_ = (df_.groupby('desc_epigrafe')['point']
       .agg(list)
       .reset_index(name='tipo_puntos')
       .merge(df_, on='desc_epigrafe',how='inner'))
df_['points_in_radius'] = df_.apply(lambda x: len([punto for punto in x['tipo_puntos'] 
                                                   if points_in_radius(x['point'],punto,radius=500)]),axis=1)-1

df_.drop(columns=['tipo_puntos','point'],inplace=True)
cl.reset(df_)
df__ = df_.copy()

In [7]:
df__.head()

,desc_epigrafe,id_local,lat,lon,points_in_radius
0,ACTIVIDADES DE LOS GIMNASIOS,10001889,40.407625,-3.697174,6
1,ACTIVIDADES DE LOS GIMNASIOS,10003512,40.429504,-3.706725,9
2,ACTIVIDADES DE LOS GIMNASIOS,20001385,40.402848,-3.713764,5
3,ACTIVIDADES DE LOS GIMNASIOS,20001865,40.404550,-3.703706,8
4,ACTIVIDADES DE LOS GIMNASIOS,20002001,40.398537,-3.701275,11


In [8]:
df__.to_csv('Data/censolocales/point_in_radius_short.csv',index=None, header=True)

In [9]:
df__.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8893 entries, 0 to 8892
Data columns (total 5 columns):
desc_epigrafe       8893 non-null object
id_local            8893 non-null int64
lat                 8893 non-null float64
lon                 8893 non-null float64
points_in_radius    8893 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 416.9+ KB
